# Generative Adversarial Networks

## 1. GAN Training
#### MNIST Dataset Generation

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Hyperparameters
batch_size = 128
discriminator_lr = 0.0002
generator_lr = 0.0002
latent_dim = 100
num_epochs = 50

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# Prepare the dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [5]:
dataset = torchvision.datasets.FashionMNIST(root="./data", train=True, transform=transform, download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

100%|██████████████████████████████████████████████████████████████████████| 26.4M/26.4M [06:32<00:00, 67.3kB/s]
100%|███████████████████████████████████████████████████████████████████████| 29.5k/29.5k [00:00<00:00, 208kB/s]
100%|██████████████████████████████████████████████████████████████████████| 4.42M/4.42M [00:03<00:00, 1.33MB/s]
100%|██████████████████████████████████████████████████████████████████████| 5.15k/5.15k [00:00<00:00, 34.8kB/s]


In [6]:
# Define the Generator
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 28 * 28),
            nn.Tanh()
        )
    
    def forward(self, z):
        img = self.model(z)
        img = img.view(img.size(0), 1, 28, 28)
        return img

In [7]:
# Define the Discriminator
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(28 * 28, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, img):
        img_flat = img.view(img.size(0), -1)
        validity = self.model(img_flat)
        return validity

In [8]:
# Initialize models
generator = Generator(latent_dim).to(device)
discriminator = Discriminator().to(device)

In [9]:
# Loss and optimizers
adversarial_loss = nn.BCELoss()
g_optimizer = optim.Adam(generator.parameters(), lr=generator_lr, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=discriminator_lr, betas=(0.5, 0.999))

In [10]:
# Fixed noise for consistent image generation
fixed_z = torch.randn(16, latent_dim).to(device)

In [11]:
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

generator.apply(weights_init)
discriminator.apply(weights_init)

Discriminator(
  (model): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=1024, out_features=512, bias=True)
    (3): LeakyReLU(negative_slope=0.2)
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): LeakyReLU(negative_slope=0.2)
    (6): Linear(in_features=256, out_features=1, bias=True)
    (7): Sigmoid()
  )
)

In [12]:
# Track losses
g_losses = []
d_losses = []

# Training loop
for epoch in range(num_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        
        real_imgs = imgs.to(device)
        batch_size = imgs.shape[0]
        valid = torch.ones(batch_size, 1).to(device)
        fake = torch.zeros(batch_size, 1).to(device)
        
        # Train Generator
        g_optimizer.zero_grad()
        z = torch.randn(batch_size, latent_dim).to(device)
        generated_imgs = generator(z)
        g_loss = adversarial_loss(discriminator(generated_imgs), valid)
        g_loss.backward()
        g_optimizer.step()
        
        # Train Discriminator
        d_optimizer.zero_grad()
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(generated_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2
        d_loss.backward()
        d_optimizer.step()
        
        g_losses.append(g_loss.item())
        d_losses.append(d_loss.item())
        
    print(f"Epoch {epoch+1}/{num_epochs} | D Loss: {d_loss.item():.4f} | G Loss: {g_loss.item():.4f}")

Epoch 1/50 | D Loss: 0.4230 | G Loss: 1.2902
Epoch 2/50 | D Loss: 0.5027 | G Loss: 0.7479
Epoch 3/50 | D Loss: 0.4464 | G Loss: 1.2065
Epoch 4/50 | D Loss: 0.4451 | G Loss: 1.1218
Epoch 5/50 | D Loss: 0.4874 | G Loss: 0.8961
Epoch 6/50 | D Loss: 0.5223 | G Loss: 1.0010
Epoch 7/50 | D Loss: 0.5402 | G Loss: 1.2279
Epoch 8/50 | D Loss: 0.4166 | G Loss: 1.3353
Epoch 9/50 | D Loss: 0.4689 | G Loss: 0.8062
Epoch 10/50 | D Loss: 0.4812 | G Loss: 1.0939
Epoch 11/50 | D Loss: 0.4897 | G Loss: 1.3759
Epoch 12/50 | D Loss: 0.5653 | G Loss: 0.9108
Epoch 13/50 | D Loss: 0.4826 | G Loss: 1.2714
Epoch 14/50 | D Loss: 0.4880 | G Loss: 1.0067
Epoch 15/50 | D Loss: 0.4917 | G Loss: 1.4964
Epoch 16/50 | D Loss: 0.5400 | G Loss: 1.4198
Epoch 17/50 | D Loss: 0.4650 | G Loss: 1.3525
Epoch 18/50 | D Loss: 0.5648 | G Loss: 1.1478
Epoch 19/50 | D Loss: 0.4587 | G Loss: 1.1906
Epoch 20/50 | D Loss: 0.5346 | G Loss: 0.8578
Epoch 21/50 | D Loss: 0.5649 | G Loss: 0.9583
Epoch 22/50 | D Loss: 0.3837 | G Loss: 1.34

In [ ]:
# Generate and visualize images after training
with torch.no_grad():
    sample_imgs = generator(fixed_z).cpu()
    grid = torchvision.utils.make_grid(sample_imgs, normalize=True)
    plt.figure(figsize=(5, 5))
    plt.imshow(grid.permute(1, 2, 0))
    plt.show()

In [ ]:
# Plot loss curves
plt.figure(figsize=(10, 5))
plt.plot(g_losses, label="Generator Loss")
plt.plot(d_losses, label="Discriminator Loss")
plt.xlabel("Iterations")
plt.ylabel("Loss")
plt.legend()
plt.title("GAN Training Losses")
plt.show()

# 2. Conditional GAN

In [ ]:
# Define the Generator class for cGAN
class cGenerator(nn.Module):
    def __init__(self, num_classes=10):
        super(cGenerator, self).__init__()
        self.label_emb = nn.Embedding(num_classes, 10)
        self.main = nn.Sequential(
            nn.Linear(100 + 10, 256),
            nn.LayerNorm(256),
            nn.ReLU(),
            nn.Linear(256, 512),
            nn.LayerNorm(512),
            nn.ReLU(),
            nn.Linear(512, 1024),
            nn.LayerNorm(1024),
            nn.ReLU(),
            nn.Linear(1024, 28 * 28),
            nn.Tanh()
        )
    
    def forward(self, noise, labels):
        label_embedding = self.label_emb(labels)
        x = torch.cat((noise, label_embedding), dim=1)
        return self.main(x).view(-1, 1, 28, 28)

In [ ]:
# Define the Discriminator class for cGAN
class cDiscriminator(nn.Module):
    def __init__(self, num_classes=10):
        super(cDiscriminator, self).__init__()
        self.label_emb = nn.Embedding(num_classes, 10)
        self.main = nn.Sequential(
            nn.Linear(28 * 28 + 10, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x, labels):
        label_embedding = self.label_emb(labels)
        x = torch.cat((x.view(-1, 28 * 28), label_embedding), dim=1)
        return self.main(x)

In [ ]:
# Initialize models
c_generator = cGenerator()
c_discriminator = cDiscriminator()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
c_generator.to(device)
c_discriminator.to(device)

In [ ]:
# Loss and optimizer
criterion = nn.BCELoss()
cg_optimizer = optim.Adam(c_generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
cd_optimizer = optim.Adam(c_discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

In [ ]:
# Load dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transform, download=True)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=128, shuffle=True)

In [ ]:
# Training loop
epochs = 20
for epoch in range(epochs):
    for i, (real_images, labels) in enumerate(dataloader):
        batch_size = real_images.size(0)
        real_images, labels = real_images.to(device), labels.to(device)

        # Train Discriminator
        real_labels = torch.ones(batch_size, 1, device=device)
        fake_labels = torch.zeros(batch_size, 1, device=device)
        
        real_outputs = c_discriminator(real_images, labels)
        cd_loss_real = criterion(real_outputs, real_labels)
        
        z = torch.randn(batch_size, 100, device=device)
        fake_images = c_generator(z, labels)
        fake_outputs = c_discriminator(fake_images.detach(), labels)
        cd_loss_fake = criterion(fake_outputs, fake_labels)
        
        cd_loss = cd_loss_real + cd_loss_fake
        cd_optimizer.zero_grad()
        cd_loss.backward()
        cd_optimizer.step()

        # Train Generator
        fake_outputs = c_discriminator(fake_images, labels)
        cg_loss = criterion(fake_outputs, real_labels)
        cg_optimizer.zero_grad()
        cg_loss.backward()
        cg_optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], D Loss: {cd_loss.item():.4f}, G Loss: {cg_loss.item():.4f}')

In [ ]:
# Generate and display a sample image of a specific digit
z = torch.randn(1, 100, device=device)
label = torch.tensor([2], device=device)  # Generate a '7'
with torch.no_grad():
    generated_image = c_generator(z, label).cpu().detach().numpy().reshape(28, 28)
plt.imshow(generated_image, cmap='gray')
plt.show()